In [1]:
import pennylane as qml
from pennylane import numpy as np

from context import QNetOptimizer as QNopt

In [32]:
arb_prep_nodes = [
    QNopt.PrepareNode(3, [0], qml.templates.subroutines.ArbitraryUnitary, 3),
    QNopt.PrepareNode(3, [1], qml.templates.subroutines.ArbitraryUnitary, 3)
]
prep_nodes = [
    QNopt.PrepareNode(3, [0], QNopt.local_RY, 1),
    QNopt.PrepareNode(3, [1], QNopt.local_RY, 1)
]

arb_meas_nodes = [
    QNopt.MeasureNode(1, 2, [0,1], qml.templates.subroutines.ArbitraryUnitary, 15)
]
local_RY_meas_nodes = [
    QNopt.MeasureNode(1, 2, [0,1], QNopt.local_RY, 2)
]

def entangled_meas(settings, wires):
    qml.CNOT(wires=wires)
    qml.RY(settings[0],wires=wires[0])
    qml.RY(settings[1],wires=wires[1])

entangled_meas_nodes = [
    QNopt.MeasureNode(1, 2, [0,1], entangled_meas, 2)
]

arb_mac_ansatz = QNopt.NetworkAnsatz(arb_prep_nodes, arb_meas_nodes)
local_RY_mac_ansatz = QNopt.NetworkAnsatz(prep_nodes, local_RY_meas_nodes)
entangled_meas_mac_ansatz = QNopt.NetworkAnsatz(prep_nodes, entangled_meas_nodes)

In [36]:
# entanglement assisted classical ansatz

entanglement_prep_nodes = [
    QNopt.PrepareNode(1, [0,1], qml.templates.subroutines.ArbitraryUnitary, 15)
]
local_arb_meas_nodes = [
    QNopt.MeasureNode(3, 2, [0], qml.templates.subroutines.ArbitraryUnitary, 3),
    QNopt.MeasureNode(3, 2, [1], qml.templates.subroutines.ArbitraryUnitary, 3),
]

entanglement_assisted_mac_ansatz = QNopt.NetworkAnsatz(entanglement_prep_nodes, local_arb_meas_nodes)

In [38]:
mac_bound_inequalities = [
    (np.array([
        [0,0,0,0,1,0,1,0,0],
        [1,1,0,1,0,0,0,0,0]
    ]), 4),
    (np.array([
        [0,0,1,0,1,0,0,0,0],
        [1,1,0,1,0,0,0,0,0],
    ]), 4),
    (np.array([
        [0,0,0,0,0,1,1,0,0],
        [1,1,0,1,0,0,0,1,0],
    ]), 5),
    (np.array([
        [0,0,0,0,1,0,0,0,1],
        [1,1,0,1,0,1,0,0,0],
    ]), 5),
    (np.array([
        [0,0,1,0,1,0,1,0,0],
        [2,1,0,1,0,1,0,1,0],
    ]), 7),
    (np.array([
        [0,0,1,0,1,0,1,0,0],
        [2,2,0,1,0,0,0,1,0],
    ]), 7),
    (np.array([
        [0,0,1,0,1,0,1,0,0],
        [2,1,0,2,0,1,0,0,0],
    ]), 7),
    (np.array([
        [0,0,0,0,1,1,1,0,1],
        [1,1,0,1,0,0,0,1,0],
    ]), 6),
    (np.array([
        [0,0,0,0,1,1,2,0,0],
        [2,2,0,1,0,0,0,1,0],
    ]), 8),
    (np.array([
        [0,0,0,0,2,0,1,0,1],
        [2,2,0,1,0,1,0,0,0],
    ]), 8),
    (np.array([
        [0,0,1,0,2,0,0,0,1],
        [2,1,0,2,0,1,0,0,0],
    ]), 8),
    (np.array([
        [0,0,1,0,2,0,0,0,1],
        [2,1,0,2,0,0,0,1,0],
    ]), 8),
    (np.array([
        [0,0,1,0,1,0,2,0,0],
        [2,1,0,0,0,1,0,2,0],
    ]), 8),
    (np.array([
        [0,0,1,0,2,0,1,0,1],
        [3,2,0,2,0,1,0,0,0],
    ]), 10),
    (np.array([
        [0,0,1,0,2,0,1,0,1],
        [3,2,0,2,0,0,0,1,0],
    ]), 10),
    (np.array([
        [0,0,2,0,1,0,2,0,0],
        [3,1,0,1,0,2,0,2,0],
    ]), 11),
    (np.array([
        [0,0,2,0,2,0,2,0,0],
        [3,1,0,1,0,3,0,3,1],
    ]), 14),
    (np.array([
        [0,0,2,0,3,0,2,0,1],
        [5,3,0,3,0,1,0,1,0],
    ]), 16),
    (np.array([
        [0,0,2,1,2,0,5,0,1],
        [4,2,0,0,0,1,0,4,0],
    ]), 17),
]

In [35]:
for (i,game_tuple) in enumerate(mac_bound_inequalities):
    
    print("\ninequality : ", i)

    settings = arb_mac_ansatz.rand_scenario_settings()

    cost = QNopt.linear_probs_cost(arb_mac_ansatz, game_tuple[0])

    opt_dict = QNopt.gradient_descent(cost, settings, step_size=0.1, num_steps=200, sample_width=25)
    
    print("\nclassical bound : ", game_tuple[1])

    print("max_score : ", opt_dict["opt_score"])


inequality :  0
iteration :  0 , score :  2.0680770430877264
iteration :  25 , score :  4.3873765013792
iteration :  50 , score :  4.40414555521024
iteration :  75 , score :  4.4097592392454334
iteration :  100 , score :  4.412223497615171
iteration :  125 , score :  4.413299344230882
iteration :  150 , score :  4.413769193775705
iteration :  175 , score :  4.413977952167734

classical bound :  4
max_score :  4.414074513867206

inequality :  1
iteration :  0 , score :  2.4524117807259955
iteration :  25 , score :  3.9707629285330355
iteration :  50 , score :  3.9973366095366645
iteration :  75 , score :  3.999489165718805
iteration :  100 , score :  3.999833820188825
iteration :  125 , score :  3.999926402834944
iteration :  150 , score :  3.999962705159278
iteration :  175 , score :  3.999980119728607

classical bound :  4
max_score :  3.9999892091447693

inequality :  2
iteration :  0 , score :  3.895419687525293
iteration :  25 , score :  5.262531882701801
iteration :  50 , score :


classical bound :  17
max_score :  17.400615480439228


In [21]:
for (i,game_tuple) in enumerate(mac_bound_inequalities):
    
    print("\n\ninequality : ", i)

    settings = arb_mac_ansatz.rand_scenario_settings()

    cost = QNopt.linear_probs_cost(local_RY_mac_ansatz, game_tuple[0])

    opt_dict = QNopt.gradient_descent(cost, settings, step_size=0.1, num_steps=200, sample_width=50)
    
    print("\n\nclassical bound : ", game_tuple[1])

    print("max_score : ", opt_dict["opt_score"])



inequality :  0
iteration :  0 , score :  2.8165875803410647
iteration :  50 , score :  3.4589910505631396
iteration :  100 , score :  3.9978059169474505
iteration :  150 , score :  3.9999968921671547


classical bound :  4
max_score :  3.9999999956015433


inequality :  1
iteration :  0 , score :  2.748766481897025
iteration :  50 , score :  3.9997451564408486
iteration :  100 , score :  3.9999998252480458
iteration :  150 , score :  3.999999999835241


classical bound :  4
max_score :  3.999999999999803


inequality :  2
iteration :  0 , score :  3.0115996360122743
iteration :  50 , score :  4.997809715236018
iteration :  100 , score :  4.999998954497033
iteration :  150 , score :  4.999999999055544


classical bound :  5
max_score :  4.999999999996317


inequality :  3
iteration :  0 , score :  2.709795852268505
iteration :  50 , score :  3.9729794081901337
iteration :  100 , score :  3.999310618069539
iteration :  150 , score :  3.9999790679322813


classical bound :  5
max_score

In [27]:
for (i,game_tuple) in enumerate(mac_bound_inequalities):
    
    print("\n\ninequality : ", i)

    settings = entangled_meas_mac_ansatz.rand_scenario_settings()

    cost = QNopt.linear_probs_cost(entangled_meas_mac_ansatz, game_tuple[0])

    opt_dict = QNopt.gradient_descent(cost, settings, step_size=0.1, num_steps=200, sample_width=50)
    
    print("\n\nclassical bound : ", game_tuple[1])

    print("max_score : ", opt_dict["opt_score"])



inequality :  0
iteration :  0 , score :  3.0894143424249068
iteration :  50 , score :  3.9910029701021523
iteration :  100 , score :  3.9994615871572625
iteration :  150 , score :  3.9999653528035073


classical bound :  4
max_score :  3.99999758412628


inequality :  1
iteration :  0 , score :  2.427959134164027
iteration :  50 , score :  4.014860412375238
iteration :  100 , score :  4.079023876538785
iteration :  150 , score :  4.278554951211023


classical bound :  4
max_score :  4.404957521665724


inequality :  2
iteration :  0 , score :  2.5348827525252426
iteration :  50 , score :  4.965253414756241
iteration :  100 , score :  4.999413449641993
iteration :  150 , score :  4.999983798865266


classical bound :  5
max_score :  4.999999537448462


inequality :  3
iteration :  0 , score :  1.557155717188104
iteration :  50 , score :  4.978015073342052
iteration :  100 , score :  5.24752898226017
iteration :  150 , score :  5.259926775302081


classical bound :  5
max_score :  5.2

In [37]:
for (i,game_tuple) in enumerate(mac_bound_inequalities):
    
    print("\n\ninequality : ", i)

    settings = entanglement_assisted_mac_ansatz .rand_scenario_settings()

    cost = QNopt.linear_probs_cost(entanglement_assisted_mac_ansatz , game_tuple[0])

    opt_dict = QNopt.gradient_descent(cost, settings, step_size=0.1, num_steps=200, sample_width=50)
    
    print("\n\nclassical bound : ", game_tuple[1])

    print("max_score : ", opt_dict["opt_score"])



inequality :  0
iteration :  0 , score :  1.7230228007165342
iteration :  50 , score :  4.410758148815941
iteration :  100 , score :  4.414121326784323
iteration :  150 , score :  4.4142079651962876


classical bound :  4
max_score :  4.414213192682524


inequality :  1
iteration :  0 , score :  1.8394737393786027
iteration :  50 , score :  4.089413574537114
iteration :  100 , score :  4.4128096250852655
iteration :  150 , score :  4.414175663184001


classical bound :  4
max_score :  4.4142045035241


inequality :  2
iteration :  0 , score :  2.27575172648964
iteration :  50 , score :  5.177028621724968
iteration :  100 , score :  5.382442146790156
iteration :  150 , score :  5.409366735929772


classical bound :  5
max_score :  5.4133904501527965


inequality :  3
iteration :  0 , score :  3.110923535221829
iteration :  50 , score :  5.199945222171486
iteration :  100 , score :  5.385788750206753
iteration :  150 , score :  5.403106610752138


classical bound :  5
max_score :  5.40

In [ ]:
a = []

In [ ]:
a += [2]

In [ ]:
a

In [ ]:
a = [1,2,3,4,5]

In [ ]:
i=1
a[i+1:]

In [ ]:
3 % 5

In [ ]:
3 // 3

In [ ]:
np.sum(game[:,1]*[1,-5])


In [ ]:
np.eye(8)